# Imports

In [1]:
import numpy as np 
import pandas as pd

# Pre-processing and acquiring the data 

## Getting the Wikipedia page

In [2]:
link = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto_data = pd.read_html(link,header=0)[0]
toronto_data = toronto_data.replace(to_replace='Not assigned',value=np.nan)
toronto_data = toronto_data.loc[toronto_data.Borough.notna(),:]
toronto_data = toronto_data.apply(lambda x: [x.Postcode,x.Borough, x.Borough] if pd.isna(x.Neighbourhood)
                       else x, axis=1)
toronto_data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Okay let's breakdown what I did here. I replaces Not assigned with a nan value and then dropped for boroughs and filled the neighboorhood names.

Next, I will conbine neighboorhoods with the same postal code as required

In [3]:
counts = toronto_data.Postcode.value_counts()
counts[counts>1]

M9V    8
M8Y    8
M5V    7
M9B    5
M8Z    5
M4V    5
M1V    4
M9R    4
M9C    4
M6M    4
M1L    3
M5T    3
M2J    3
M1K    3
M3H    3
M8X    3
M1T    3
M1M    3
M5J    3
M1C    3
M8V    3
M6L    3
M5H    3
M5R    3
M1P    3
M6K    3
M1E    3
M6H    2
M5L    2
M5K    2
M5P    2
M1N    2
M5B    2
M9M    2
M5M    2
M3J    2
M1R    2
M4X    2
M4T    2
M4L    2
M6R    2
M8W    2
M5X    2
M2L    2
M4K    2
M2M    2
M5A    2
M6P    2
M3K    2
M1B    2
M5S    2
M4B    2
M6N    2
M6S    2
M6J    2
M3C    2
M6A    2
Name: Postcode, dtype: int64

We need to combine these neighboorhood names separated by commas 

In [4]:
toronto_data = toronto_data.groupby('Postcode').apply(lambda x: x.apply(lambda y:  ', '.join(y) \
                                                   if y.name =='Neighbourhood' \
                                                   else y.tolist()[0]))
toronto_data.reset_index(inplace=True, drop=True)
toronto_data.loc[toronto_data.Postcode.isin(counts[counts>1].index),:].head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [5]:
toronto_data.loc[toronto_data.Postcode.isin(counts[counts==1].index),:].head()

,Postcode,Borough,Neighbourhood
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
12,M1S,Scarborough,Agincourt
15,M1W,Scarborough,L'Amoreaux West


We demonstrated that our code worked well

In [6]:
# The shape of the table 
toronto_data.shape

(103, 3)

In [7]:
toronto_data.to_csv('toronto_data.csv', index=False)